In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/My\ Drive/dataset
!ls
!pwd

In [ ]:
!mkdir val_images

In [ ]:
!ls

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
%cd train_images

In [ ]:
import os
import random
 
root_dir = '.'

img_path_list = []
possible_img_extension = ['.jpg', '.PNG'] # 이미지 확장자들
 
for (root, dirs, files) in os.walk(root_dir):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                img_path = root + '/' + file_name
                
                # 경로에서 \를 모두 /로 바꿔줘야함
                img_path = img_path.replace('\\', '/') # \는 \\로 나타내야함 
                img_path = img_path[2:]        
                img_path_list.append(img_path)

img_path_list.sort()

#print(img_path_list)
# img_path_list = os.listdir('.')
# print(sorted(img_path_list))

# dataset_dicts = get_train_dicts()

val_samples = 0
# random.Random(1000).shuffle(img_path_list)
# random.Random(1000).shuffle(dataset_dicts)
val_images = sorted(random.sample(img_path_list, val_samples))

# train_input_img_paths = img_path_list[:-val_samples]
# train_dicts = dataset_dicts[:-val_samples]
# val_input_img_paths = img_path_list[-val_samples:]
# val_dicts = dataset_dicts[-val_samples:]

start = './'
destination = '../val_images/'
for image in val_images:
  os.replace(start + image, destination + image)
print(len(os.listdir(destination)))


In [ ]:
print(img_path_list)

In [ ]:
# 파일 원래대로 돌리기
for image in os.listdir(destination):
  os.replace(destination + image, start + image)
print('destination : {}, start : {}'.format(len(os.listdir(destination)), len(os.listdir(start))))

In [ ]:
from detectron2.structures import BoxMode

def get_train_dicts(dir):
  json_file = os.path.join('..', 'train_answer.json')
  with open(json_file) as f:
    imgs_anns = json.load(f)

  dataset_dicts = []
  objects = []

  for _, v in enumerate(imgs_anns['annotations']):
    if objects == []:
      v = [v]
      objects.append(v)
      temp = 0
    elif v['image_id'] > objects[temp][0]['image_id']:
      v = [v]
      objects.append(v)
      temp += 1
    else:
      objects[temp].append(v)

  for idx, v in enumerate(imgs_anns['images']):
    record = {}

    record['file_name'] = v['file_name']
    record['image_id'] = v['id']
    record['height'] = v['height']
    record['width'] = v['width']

    annos = objects[idx]
    objs = []

    for anno in annos:
      poly = anno['segmentation']
      px = []
      py = []
      for idx, pos in enumerate(poly):
        if idx % 2 == 0:
          px.append(pos)
        else:
          py.append(pos)
          
      obj = {
          'bbox': [np.min(px), np.min(py), np.max(px), np.max(py)],
          'bbox_mode': BoxMode.XYXY_ABS,
          'segmentation': [poly],
          'category_id': anno['category_id']-1,
      }

      objs.append(obj)

    record['annotations'] = objs
    dataset_dicts.append(record)

  val_images = os.listdir('../val_images')
  
  data1 = dataset_dicts[:2000]
  data2 = dataset_dicts[2000:]

  if dir == '../train_images':       
    for i in data1:
      if i['file_name'] in val_images:
        dataset_dicts.remove(i)

  elif dir == '../val_images':
    for i in data1:
      if i['file_name'] not in val_images:
        dataset_dicts.remove(i)


  if dir == '../train_images':
    for i in data2:
      if i['file_name'] in val_images:
        dataset_dicts.remove(i)

  elif dir == '../val_images':
    for i in data2:
      if i['file_name'] not in val_images:
        dataset_dicts.remove(i)

  return dataset_dicts

In [ ]:
print(len(os.listdir('../val_images')))
print(len(os.listdir('../train_images')))
print(len(get_train_dicts('../train_images')))
print(len(get_train_dicts('../val_images')))


In [ ]:
for d in ['train', 'val']:
  DatasetCatalog.register(d + '_images', lambda d=d: get_train_dicts('../' + d + '_images'))
  MetadataCatalog.get(d + '_images').set(thing_classes=['cow', 'cow1'])


# for d in ['train', 'val']:
#   if d == 'train':
#     # DatasetCatalog.register(d + '_images', lambda d=d: get_train_dicts('../' + d + '_images'))
#     print('d')
#   else:
#     # DatasetCatalog.register(d + '_images', lambda d=d: get_train_dicts('../' + d + '_images'))
#     print('d')
#   MetadataCatalog.get(d + '_images').set(thing_classes=['no', 'cow', 'cow1'])

# DatasetCatalog.register('train_images', get_train_dicts)
# MetadataCatalog.get('train_images').set(thing_classes=['no', 'cow', 'cow1'])

cow_metadata = MetadataCatalog.get('train_images')
print(cow_metadata)

In [ ]:
dataset_dicts = get_train_dicts('../train_images')
for d in random.sample(dataset_dicts, 3):
  img = cv2.imread(d['file_name'])
  visualizer = Visualizer(img[:, :, ::-1], metadata = cow_metadata, scale=0.5)
  out = visualizer.draw_dataset_dict(d)
  cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
!pwd

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
cfg.DATASETS.TRAIN = ('train_images',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 10
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
%cd train_images/

In [ ]:
%cd ..

In [ ]:
!pwd

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode

dataset_dicts = get_train_dicts('../train_images')
for d in random.sample(dataset_dicts, 5):
  img = cv2.imread('../train_images/'+d['file_name'])
  print(d['file_name'])
  outputs = predictor(img)
  visualizer = Visualizer(img[:, :, ::-1], metadata = cow_metadata, scale=0.5, instance_mode = ColorMode.IMAGE_BW)
  out = visualizer.draw_instance_predictions(outputs['instances'].to('cpu'))
  cv2_imshow(out.get_image()[:, :, ::-1])
  print(outputs)

# json 파일 출력하는법 + test_image모두 돌리는거
### 각 셀마다 실행하기 전에 경로지정 잘해야됨

In [ ]:
import skimage
from skimage import measure

def close_contour(contour):
  if not np.array_equal(contour[0], contour[-1]):
    contour = np.vstack((contour, contour[0]))
  return contour

def binary_mask_to_polygon(binary_mask, tolerance=0):
  polygons = []
  # pad mask to close contours of shapes which start and end at an edge
  padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
  contours = measure.find_contours(padded_binary_mask, 0.5)
  contours = np.subtract(contours, 1)
  for contour in contours:
    contour = close_contour(contour)
    contour = measure.approximate_polygon(contour, tolerance)
    if len(contour) < 3:
      continue
    contour = np.flip(contour, axis=1)
    segmentation = contour.ravel().tolist()
    # after padding and subtracting 1 we may get -0.5 points in our segmentation
    segmentation = [0 if i < 0 else i for i in segmentation]
    polygons.append(segmentation)
  return polygons

In [ ]:
!pwd #아마 train_images로 되있을꺼임

In [ ]:
%cd ..

In [ ]:
!pwd

In [ ]:
%cd test_images 
#테스트할 이미지의 test_image_path_list를 만들어주기 위해 test_images폴더로 이동

In [ ]:
test_dir = '.'

test_img_path_list = []
 
for (root, dirs, files) in os.walk(test_dir):
    if len(files) > 0:
        for file_name in files:
            if os.path.splitext(file_name)[1] in possible_img_extension:
                test_img_path = root + '/' + file_name
                
                # 경로에서 \를 모두 /로 바꿔줘야함
                test_img_path = test_img_path.replace('\\', '/') # \는 \\로 나타내야함 
                test_img_path = test_img_path[2:]        
                test_img_path_list.append(test_img_path)

test_img_path_list.sort()

#print(test_img_path_list[:2])

In [ ]:
!pwd #test_imgaes인거 경로 확인 하고 아래 실행해야함

In [ ]:
file_path = "../test_answer.json"

test_data = {}
test_data["images"] = []
test_data["annotations"] = []

for d in test_img_path_list:
  timg = cv2.imread(d)
  #cv2_imshow(timg)
  h, w, c = timg.shape
  test_data["images"].append({
      "file_name": d,
      "width": int(w),
      "height": int(h),
      "id": int(d[5:9])
  })
  outputs = predictor(timg)
  pred_masks_tensor = outputs['instances'].get_fields()['pred_masks']
  pred_classes = outputs['instances'].get_fields()['pred_classes']
  classes = pred_classes.cpu().numpy()
  mask = pred_masks_tensor.cpu().numpy()
  mask_array = outputs["instances"].pred_masks.cpu().numpy()
  num_instances = mask_array.shape[0]
  for a in range(num_instances):
    poly=binary_mask_to_polygon(mask[a])
    test_data['annotations'].append({
        "image_id": int(d[5:9]),
        "segmentation": poly[0],
        "category_id": int(classes[a] + 1),
        "conf": float(0.0)
    })


with open(file_path, 'w') as outfile:
  json.dump(test_data, outfile)

In [ ]:
cd .. #json 출력 확인해보기 - 상위디렉토리 이동

In [ ]:
!cat test_answer.json #json파일 열어보기